In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import tifffile
import rasterio

In [2]:
def read_data(file_path):
    latitudes = []
    longitudes = []
    swir2 = []
    swir = []
    nnir = []
    red = []
    green = []
    blue = []
    
    with open(file_path, 'r') as file:
        for line in file:
            lat, lon, s2, s, n, r, g, b = map(float, line.split())
            latitudes.append(lat)
            longitudes.append(lon)
            swir2.append(s2)
            swir.append(s)
            nnir.append(n)
            red.append(r)
            green.append(g)
            blue.append(b)
    
    return latitudes, longitudes, swir2, swir, nnir, red, green, blue

# File path containing the data
file_path_01012015 = "/home/hzin/earthAI/logs/140101.txt"  # Change this to your file path

# Read data from the file
latitudes, longitudes, swir2, swir, nnir, red, green, blue = read_data(file_path_01012015)

In [3]:
# Define the grid boundaries and resolution
min_lat = min(latitudes)
max_lat = max(latitudes)
min_lon = min(longitudes)
max_lon = max(longitudes)
resolution = 0.05  # Adjust as needed

# Generate grid points
grid_lat, grid_lon = np.meshgrid(
    np.arange(min_lat, max_lat, resolution),
    np.arange(min_lon, max_lon, resolution)
)

# Perform interpolation using inverse distance weighting
interpolated_grids = []
for var in [blue, green, red, nnir, swir, swir2]:
    interpolated_grid = griddata(
        (latitudes, longitudes),
        var,
        (grid_lat, grid_lon),
        method='nearest'
    )
    interpolated_grids.append(interpolated_grid)

combined_grid = np.stack(interpolated_grids, axis=-1).transpose(2, 1, 0)
print(combined_grid.shape)



(6, 440, 400)


Rescale

In [4]:
# Find the minimum and maximum values along the first three dimensions
min_vals = np.min(combined_grid, axis=(1, 2), keepdims=True)
max_vals = np.max(combined_grid, axis=(1, 2), keepdims=True)

# Rescale the values
rad_scaled = ((combined_grid - min_vals) / (max_vals - min_vals)) * 255

# Convert rescaled values to integers
rad_scaled = rad_scaled.astype(np.uint8)

In [35]:
# Save the combined grid to a TIFF file
output_tiff_file = "140101.tiff"
tifffile.imwrite(output_tiff_file, rad_scaled, photometric='rgb')


In [36]:
def load_raster(path, crop=None):
    with rasterio.open(path) as src:
        img = src.read()

        # load first 6 bands
        img = img[:6]
    return img
#input_data = load_raster("140101.tiff")
input_data = tifffile.imread("140101.tiff")
print(input_data.shape)

(6, 440, 400)
